In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os
join = os.path.join
from tqdm import tqdm
import nibabel as nib
from skimage import exposure


def show_img_label(img_array, roi_array, show_index=None):
    # plt.clf()
    if show_index != None:
        show_img = img_array[show_index, ...]
        show_roi = roi_array[show_index, ...]
    else:
        show_img = img_array
        show_roi = roi_array
    # plt.axis('off')
    plt.imshow(show_img, cmap='gray')
    plt.contour(show_roi, linewidths=0.3)
    
def show_from_path(img_path, roi_path=None, trans=True):
    size = "small"
    img = nib.load(img_path)
    img_arr = img.get_fdata()
    exposure.rescale_intensity(img_arr, in_range=(-1250, 500), out_range=(0,1))
    if roi_path is not None:
        roi = nib.load(roi_path)
        roi_arr = roi.get_fdata()
        if np.max(roi_arr) > 1: 
            roi_arr[roi_arr==1] = 0
            roi_arr[roi_arr==2] = 1
        if trans:
            roi_arr = roi_arr
        if np.sum(roi_arr) >= 30:
            size = "big"   
    show_img_label(img_arr, roi_arr)
    return size

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='red', facecolor=(0,0,0,0), lw=0.3)) 
    
def get_box(data):
    y_indices, x_indices = np.where(data > 0)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)

    box_np = np.array([x_min, y_min, x_max, y_max])
    return box_np 
    
def show_mask_pred(img_path, mask_path, pred_path,gt_path=None):
    plt_num = 2
    if gt_path is not None:
        plt_num = 3
        plt.subplot(1, plt_num, 3)
        plt.title("gt")
        show_from_path(img_path, gt_path)
    plt.subplot(1,plt_num,1)
    plt.title("before refine")
    size = show_from_path(img_path, mask_path, False)
    plt.subplot(1,plt_num,2)
    plt.title("pred")
    show_from_path(img_path, pred_path)        
    return size


In [ ]:

img_root = "/data3/home/lishengyong/data/ssc_0802/new821/images"
mask_root = "/data3/home/lishengyong/data/ssc_0802/new821/masks"
gt_root = "/data3/home/lishengyong/data/ssc_0802/new821/labels"
infer_root = "/data3/home/lishengyong/data/ssc_0802/new821/infer1/0818_98_processed"
infer_root1 = "/data3/home/lishengyong/data/ssc_0802/slices/infer/0807_onebox"
check_out_path = "/data3/home/lishengyong/data/ssc_0802/new_slices/check/old"
os.makedirs(check_out_path, exist_ok=True)
cases = os.listdir(mask_root)
for case_name in cases[:20]:
        #show_from_path(join(img_root, case_name), join(mask_root, case_name))
        show_mask_pred(join(img_root, case_name), join(mask_root, case_name), join(infer_root, case_name))
        # plt.subplot(144)
        # plt.title("one-box")
        # show_from_path(join(img_root, case_name), join(infer_root1, case_name))
        # plt.savefig(join(check_out_path, case_name[:-7])+".png", dpi=300)
        plt.show()

In [39]:
# stack 2d
root = "/data3/home/lishengyong/data/ssc_0802/new821"
img_root = join(root, "images")
mask_root = join(root, "masks")
gt_root = join(root, "labels")
infer_root = join(root, "infer", "0818_98_processed")
store_path = join(root, "check")
cases = os.listdir(infer_root)
cases.sort()
def read_img(imgs, img_path):
    img = nib.load(img_path)
    img_arr = img.get_fdata()
    return imgs.append(img_arr)
def write_img(imgs, key):
    img = np.stack(imgs)
    nib.save(nib.Nifti1Image(img, affine=None), join(store_path, f"{key}.nii.gz"))
imgs, masks, gts, preds = [], [], [], []
cases = [f"{i}.nii.gz" for i in range(500)]
for i in cases:
    for j, path in zip([imgs, masks, preds], [img_root, mask_root, infer_root]):
        j = read_img(j, join(path, i))
for j, path in zip([imgs, masks, preds], ["images", "masks", "preds"]):
    write_img(j, path)

In [ ]:
import h5py
embaddings = [i for i in Path("/data3/home/lishengyong/data/ssc_0802/slices/infer_3d/axis_1/0807_1536/iter2/embaddding").glob("*_0.h5py")]
for embadding in embaddings[:10]:
    with h5py.File(embadding) as file:
        ax = plt.gca()
        mask = nib.load(str(embadding).replace("_0.h5py", ".nii.gz").replace("embadding", "masks"))
        mask_arr = mask.get_fdata().T
        plt.imshow(file["label"][0])
        show_box(file["bbox"], ax)
        plt.show()

In [ ]:
# show 3d
import matplotlib.pyplot as plt 
from pathlib import Path
import SimpleITK as sitk
masks = [i for i in Path("/data3/home/lishengyong/data/ssc_0802/slices/3D_masks").iterdir()]
for i in masks[40:60]:
    roi = nib.load(i)
    roi_arr = roi.get_fdata()
    plt.subplot(131).imshow(roi_arr[64])
    plt.subplot(132).imshow(roi_arr[:,64,:])
    plt.subplot(133).imshow(roi_arr[..., 16])
    plt.show()

In [ ]:
masks = [i for i in Path("/data3/home/lishengyong/data/ssc_0802/slices/infer_3d/axis_1/0807_1536").iterdir()]
masks.sort()
for j in range(10):
    mask = str(masks[j])
    roi = nib.load(mask)
    roi_arr = roi.get_fdata()(roi)
    img = nib.load(mask.replace("infer_3d/axis_1/0807_1536", "3D_images"))
    img_arr = img.get_fdata()(img)
    bf_refine = nib.load(mask.replace("infer_3d/axis_1/0807_1536", "3D_masks"))
    bf_refine_arr = bf_refine.get_fdata()(bf_refine)
    k = 1
    plt.figure(figsize=(10, 40))
    for i in range(12, 20):
        plt.subplot(8, 2, 2 * k - 1)
        plt.imshow(img_arr[..., i])
        plt.contour(bf_refine_arr[..., i])
        plt.axis("off")
        plt.subplot(8, 2, 2 * k)
        plt.imshow(img_arr[..., i])
        plt.contour(roi_arr[..., i])
        plt.axis("off")
        k += 1
    plt.subplots_adjust(wspace=0)
    plt.show()
    # plt.savefig(f"/data3/home/lishengyong/data/ssc_0802/slices/check/{masks[j].name.replace('nii.gz', 'png')}", dpi=300)
    plt.close()

In [56]:
trans_img_path = "/data3/home/lishengyong/data/ssc_0802/slices/infer_3d/axis_1/trans_images"
trans_mask_path = "/data3/home/lishengyong/data/ssc_0802/slices/infer_3d/axis_1/trans_masks"
imgs, masks, gts, preds = [], [], [], []
for i in cases:
    for j, path in zip([imgs, masks], [trans_img_path, trans_mask_path]):
        j = read_img(j, join(path, i))
for j, path in zip([imgs, masks], ["trans_images", "trans_masks"]):
    write_img(j, path)

In [11]:
import json
with open("/data3/home/lishengyong/data/ssc_3d/slices/jsons/lndb_weight.json", "r") as json_file:
    cases = json.load(json_file) 

axis = 0
img_path = "/data3/home/lishengyong/data/ssc_3d/slices/3D_images"
mask_path = "/data3/home/lishengyong/data/ssc_3d/slices/3D_masks"
pred_path = f"/data3/home/lishengyong/data/ssc_3d/slices/infer_3d_iter/axis_0/0818_98/jiehe"
check_path = f"/data3/home/lishengyong/data/ssc_3d/slices/infer_3d/check_out/new825/0020/axis0"
os.makedirs(check_path, exist_ok=True)
cases.sort()
imgs, masks, preds = [], [], []
for case in cases[:20]:
    case = case+".nii.gz"
    if not os.path.exists(join(pred_path, case)):
        print("no exist", case)
        continue
    pred = nib.load(join(pred_path, case))
    pred_arr = pred.get_fdata()
    if np.max(pred_arr) == 0:
        print(case)
    preds.append(pred_arr)
    img = nib.load(join(img_path, case))
    img_arr = img.get_fdata()
    imgs.append(img_arr)
    mask = nib.load(join(mask_path, case))
    mask_arr = mask.get_fdata()
    masks.append(mask_arr)

img = np.concatenate(imgs, axis=axis)
nib.save(nib.Nifti1Image(img, affine=None), join(check_path, "check_imgs.nii.gz"))
mask = np.concatenate(masks, axis=axis)
nib.save(nib.Nifti1Image(mask, affine=None), join(check_path, "check_masks.nii.gz"))
pred = np.concatenate(preds, axis=axis)
nib.save(nib.Nifti1Image(pred, affine=None), join(check_path, f"pred_axis{axis}.nii.gz"))

FileNotFoundError: [Errno 2] No such file or directory: '/data3/home/lishengyong/data/ssc_3d/slices/lndb_weight.json'

In [9]:
# stack trans
pred_path = f"/data3/home/lishengyong/data/ssc_0802/new_slices/infer_3d/axis_1/0818_98/iter8/single_bbox"
img_path = "/data3/home/lishengyong/data/ssc_3d/slices/infer_3d_iter/trans_images"
check_path = f"/data3/home/lishengyong/data/ssc_0802/new_slices/infer_3d/check_out/new825/trans1"
os.makedirs(check_path, exist_ok=True)
axis = 1
imgs, masks, preds = [], [], []
for case in cases[:20]:
    case_img = []
    for slice_index in range(128):
        slice_path = join(pred_path, case[:-7], f"{slice_index}_0.nii.gz")
        if os.path.exists(slice_path):
            slice_img = nib.load(slice_path).get_fdata()
        else:
            slice_img = np.zeros((128, 128))
        case_img.append(slice_img)
    roi3d = np.stack(case_img, axis)
    preds.append(roi3d)
    img = nib.load(join(img_path, case))
    img_arr = img.get_fdata()
    imgs.append(img_arr)

img = np.concatenate(imgs, axis=axis)
nib.save(nib.Nifti1Image(img, affine=None), join(check_path, "check_imgs.nii.gz"))
pred = np.concatenate(preds, axis=axis)
nib.save(nib.Nifti1Image(pred, affine=None), join(check_path, f"pred_axis{axis}_aug.nii.gz"))
            
            